In [1]:
import sys
import os
diretorio_atual = os.getcwd()
sys.path.append(os.path.join(diretorio_atual, '..', '..'))

import geopandas as gpd
from os.path import join
from unidecode import unidecode

from utils import (
    save_parquet,
)
from utils.downloads import (
    _prepare_cache, 
    download_malha_geosampa,
)

C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap\notebooks


# Declarar variáveis

In [2]:
gdf_distritos_ibge = gpd.GeoDataFrame()
gdf_distritos = gpd.GeoDataFrame()

# IBGE

## Importar dados 

Inicialmente, a malha dos distritos vinha do geosampa. Contudo, por conta das incompatibilidades das malhas e, principalmente, como a malha do IBGE já vem com a informação dos km^2 e população, optei por utilizá-la.

(Porém, ao final deste notebook ainda há o carregamento da malha do geosampa)

In [3]:
gdf_distritos_ibge = _prepare_cache(
    url='http://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/shp/UF/SP/SP_distritos_CD2022.zip',
    name_feature= 'distritos_ibge'
)

No caso dos distritos do IBGE, é preciso pegar apenas os da cidade de São Paulo, já que vêm os do Estado inteiro.

In [4]:

gdf_distritos_ibge = (
    gdf_distritos_ibge
    .loc[gdf_distritos_ibge['CD_MUN']=='3550308']
)

## Padronizaçã de colunas

### Nome das colunas

Originalmente, muitas das colunas dos gdfs que serão interseccionados têm nomes repetidos, ou estavam vazias. Para que isso não faça com que a análise das tabelas seja confusa, apagaremos as colunas que julgadas desnecessárias, e renomearemos as necessárias.

In [5]:
drop_cols = [f'v000{i}' for i in range(3,8)]
gdf_distritos_ibge = gdf_distritos_ibge.drop(columns=drop_cols)

In [6]:
rename_distrito_ibge_cols={
    'v0001' : 'total_pop', 
    'v0002' : 'total_dom'
}

gdf_distritos_ibge.rename(rename_distrito_ibge_cols, axis=1, inplace=True)

### Conteúdo das colunas

Quando formos realizar operações entre distritos e outras malhas, caso a outra malha não tenha a informação do CD_DIST, as comparações precisarão ser feitas a partir da coluna 'NM_DIST'. Portante seu conteúdo deve ser padronizado também:

In [7]:
gdf_distritos_ibge['NM_DIST']=gdf_distritos_ibge['NM_DIST'].str.upper()


In [8]:
for value in gdf_distritos_ibge['NM_DIST']:
    (
        gdf_distritos_ibge
        .loc[gdf_distritos_ibge['NM_DIST'] == value, 'NM_DIST']
    ) = unidecode(value)

Alguns dos valores que precisam ser inteiros estão constando como str e precisam ser padronizados também:

In [9]:
gdf_distritos_ibge['total_pop'] = gdf_distritos_ibge['total_pop'].astype(int)

In [10]:
gdf_distritos = download_malha_geosampa('distrito_municipal', False)

Erro ao processar o arquivo: [Errno 2] No such file or directory: 'C:\\Users\\x526378\\Desktop\\projetos\\cebrap\\coleta_cebrap\\notebooks\\data\\cache\\temporary_cache\\distrito_municipal.zip'
Tentando novamente em 0 segundos...
Erro ao processar o arquivo: [Errno 2] No such file or directory: 'C:\\Users\\x526378\\Desktop\\projetos\\cebrap\\coleta_cebrap\\notebooks\\data\\cache\\temporary_cache\\distrito_municipal.zip'
Tentando novamente em 1 segundos...
Erro ao processar o arquivo: [Errno 2] No such file or directory: 'C:\\Users\\x526378\\Desktop\\projetos\\cebrap\\coleta_cebrap\\notebooks\\data\\cache\\temporary_cache\\distrito_municipal.zip'
Tentando novamente em 2 segundos...
Erro ao processar o arquivo: [Errno 2] No such file or directory: 'C:\\Users\\x526378\\Desktop\\projetos\\cebrap\\coleta_cebrap\\notebooks\\data\\cache\\temporary_cache\\distrito_municipal.zip'
Tentando novamente em 3 segundos...
Erro ao processar o arquivo: [Errno 2] No such file or directory: 'C:\\Users\\x5

DataSourceError: '/vsizip/C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap\notebooks\data\cache\distrito_municipal.zip' does not exist in the file system, and is not recognized as a supported dataset name.

## Padronizaçã de colunas

In [ ]:
drop_distritos_cols={
    'cd_identif',
    'cd_identi0',
    'cd_usuario',
    'dt_criacao',
    'dt_atualiz'
}

gdf_distritos.drop(columns=drop_distritos_cols, axis=1, inplace=True)

# Salvar GDFS

In [ ]:
save_parquet(
    gdf_distritos,
    'distrito_geosampa',
    'assets'
)

In [ ]:
save_parquet(
    gdf_distritos_ibge,
    'distrito_ibge',
    'assets'
)